# Interactive Plots in Jupyter Notebook

It was built using the examples from [voila-gallery](https://github.com/pbugnion/voila-gallery/blob/master/country-indicators/index.ipynb)

In [73]:
from pathlib import Path

import pandas as pd
from bqplot import (Axis, Lines, LinearScale, Scatter, Tooltip, Figure)
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np

In [74]:
PREDICTIONS = Path('./age_prediction_exp_data.h5')

all_predictions = pd.read_hdf(PREDICTIONS, key='predictions')
mae_cv = pd.read_hdf(PREDICTIONS, key='regression')

In [75]:
class App:
    _toggle_button_options = ('STD', 'Ideal', 'Show all', 'Hide all')
    
    def __init__(self, predictions, mae_cv):
        self._df = predictions
        self._mae_cv = mae_cv
        
        available_models = list(all_predictions.columns)
        available_models.remove('age')
        available_models.remove('fold_idx')
        self._x_dropdown = self._create_indicator_dropdown(available_models, 0,
                                                           description='Models:')
        self._plot_container = widgets.Output()
        self._year_slider, year_slider_box = self._create_year_slider(
            self._df.age.min(), self._df.age.max())
        self._toggle_buttons = self._create_toggle_buttons()
        _app_container = widgets.VBox([
            self._x_dropdown,
            self._toggle_buttons,
            self._plot_container,
            year_slider_box
        ], layout=widgets.Layout(align_items='center', flex='3 0 auto'))
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    '<h3 style="text-align: center">Age Prediction for Different Models</h3>'
                ), 
                layout=widgets.Layout(margin='0 0 1em 0')
            ),
            _app_container
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()

    def _create_indicator_dropdown(self, indicators, initial_index, description=None):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index],
                                   description=description, layout=widgets.Layout(width='65%'))
        dropdown.observe(self._on_change, names=['value'])
        return dropdown
    
    def _create_year_slider(self, min_year, max_year):
        year_slider_label = widgets.Label('Chronological Age Range: ')
        year_slider = widgets.IntRangeSlider(
            value=[min_year, max_year],
            min=min_year, max=max_year,
            layout=widgets.Layout(width='500px'),
            continuous_update=False
        )
        year_slider.observe(self._on_change, names=['value'])
        year_slider_box = widgets.HBox([year_slider_label, year_slider])
        return year_slider, year_slider_box
    
    def _create_toggle_buttons(self):
        toggle_buttons = widgets.ToggleButtons(
            options=self._toggle_button_options,
            disabled=False,
            value=self._toggle_button_options[2],
            button_style='info',
            layout=widgets.Layout(margin='10px 0 0 0'))
        toggle_buttons.observe(self._on_change, names=['value'])
        return toggle_buttons
    
    def _create_plot(self, key, year_range, toggle_value):
        sliced_pred = self._df[self._df.age.between(*year_range)]
        sliced_pred = sliced_pred[key].dropna()
        y = self._df.loc[sliced_pred.index].age
        y_pred = sliced_pred.values
        
        mae = self._mae_cv.loc[key].mean()
        std = self._mae_cv.loc[key].std()
        
        
        sc_x = LinearScale()
        sc_y = LinearScale()

        name_tt = Tooltip(fields=['name'], show_labels=False)

        scatt = Scatter(x=y, y=y_pred, marker='circle',
                        unhovered_style={'opacity': 0.5},
                        stroke='black', scales={'x': sc_x, 'y': sc_y},
                        names=self._df.index, display_names=False)
        ax_x = Axis(scale=sc_x, label='Chronological Age (Years)')
        ax_y = Axis(scale=sc_y, orientation='vertical', label='Predicted Age (Years)')

        stroke_width = 4
        
        line_ideal = Lines(x=[y.min(), y.max()], y=[y.min(), y.max()],
                           scales= {'x': sc_x, 'y': sc_y}, colors=['green'],
                           labels=['Ideal Prediciton'], stroke_width=stroke_width)

        fig_elements = [scatt]
        
        if toggle_value == self._toggle_button_options[0]:
            line_low = Lines(x=[y.min(), y.max()], y=[y.min() - mae, y.max() - mae],
                         scales= {'x': sc_x, 'y': sc_y}, line_style='dashed', colors=['red'],
                        labels=['Ideal Prediciton - STD'], stroke_width=stroke_width)
            line_high = Lines(x=[y.min(), y.max()], y=[y.min() + mae, y.max() + mae],
                         scales= {'x': sc_x, 'y': sc_y}, line_style='dashed', colors=['red'],
                         labels=['Ideal Prediciton + STD'], stroke_width=stroke_width)
            fig_elements.append(line_low)
            fig_elements.append(line_high)
        elif toggle_value == self._toggle_button_options[1]:
            line_ideal = Lines(x=[y.min(), y.max()], y=[y.min(), y.max()],
                           scales= {'x': sc_x, 'y': sc_y}, colors=['green'],
                           labels=['Ideal Prediciton'], stroke_width=stroke_width)
            fig_elements.append(line_ideal)
        elif toggle_value == self._toggle_button_options[2]:
            line_low = Lines(x=[y.min(), y.max()], y=[y.min() - mae, y.max() - mae],
                         scales= {'x': sc_x, 'y': sc_y}, line_style='dashed', colors=['red'],
                        labels=['Ideal Prediciton - STD'], stroke_width=stroke_width)
            line_high = Lines(x=[y.min(), y.max()], y=[y.min() + mae, y.max() + mae],
                         scales= {'x': sc_x, 'y': sc_y}, line_style='dashed', colors=['red'],
                         labels=['Ideal Prediciton + STD'], stroke_width=stroke_width)
            line_ideal = Lines(x=[y.min(), y.max()], y=[y.min(), y.max()],
                           scales= {'x': sc_x, 'y': sc_y}, colors=['green'],
                           labels=['Ideal Prediciton'], stroke_width=stroke_width)
            fig_elements.append(line_low)
            fig_elements.append(line_high)
            fig_elements.append(line_ideal)
        elif toggle_value == self._toggle_button_options[3]:
            pass

        for e in fig_elements:
            e.tooltip = name_tt

        fig = Figure(marks=fig_elements, axes=[ax_x, ax_y],
                     animation_duration=100)
        fig.title = f'MAE = {mae:.2f}, STD = {std:.2f}'
        
        return fig

    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):
        x_indicator = self._x_dropdown.value
        year_range = self._year_slider.value
        toggle_value = self._toggle_buttons.value
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            fig = self._create_plot(x_indicator, year_range, toggle_value)
            display(fig)

In [76]:
app = App(all_predictions, mae_cv)
# Plot the predicted age versus the chronological age
app.container